In [130]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
from multiprocessing import Pool
from sklearn.feature_extraction.text import CountVectorizer

In [20]:
urls = []
page_num = 1
while True:
    r = requests.get("http://devpost.com/software/popular?query=is%3Afeatured&page=" + str(page_num))
    soup = BeautifulSoup(r.text, "lxml")
    projects = soup.findAll("div", {"class": "gallery-item"})
    if len(projects) == 0:
        break
    for i in projects:
        urls.append(i.find("a")['href'])
    page_num += 1

In [99]:
def parse_post(url):    
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "lxml")
    header = soup.find("header", {"id": "software-header"})
    title = header.find("h1", {"id": "app-title"}).text
    subtitle = ''
    subtitle_tag = header.find("p", {"class": "large"})
    if subtitle_tag is not None:
        subtitle = subtitle_tag.text
    like_button = header.find("a", {"class": "like-button"})
    like_count = like_button.find("span", {"class": "side-count"}).text
    built_with = soup.find("div", {"id": "built-with"})
    categories = []
    if built_with is not None:
        for i in built_with.findAll("li"):
            span = i.find("span")
            if span is not None:
                categories.append(span.text)
        
    software_lh = soup.find("div", {"class": "software-list-content"})
    hackathon = ''
    winnings = []
    if software_lh is not None:
        hackathon = software_lh.find("a").text
        winnings = []
        prizes = software_lh.findAll("li")
        for i in prizes:
            winnings.append(i.text)
        
    return {
        "title": title,
        "subtitle": subtitle,
        "like_count": like_count,
        "categories": categories,
        "hackathon": hackathon,
        "winnings": winnings
    }

In [101]:
p = Pool(5)
data = p.map(parse_post, urls)
    
with open("data.json", "w") as f:
    f.write(json.dumps(data))

In [102]:
df = pd.read_json("./data.json")

In [105]:
df.sort_values("like_count", ascending=False)

,categories,hackathon,like_count,subtitle,title,winnings
113,[python],HackRice 2016,89,\n Make Python great again\n,TrumpScript,[\nWinner\n Indeed: Most Artist...
419,"[chrome-app, arduino, serial-api, hardware, ma...",,69,\n Call Serena with press of a button\n,Call Serena Button,[]
256,"[azure, ios, xcode, gmail, mongodb, python, am...",LA Hacks 2015,59,"\n Effortless, fire-and-forget Amazon refun...",Kickback,[\nWinner\n Second Place\n ...
364,"[ace, go, guacamole, html5, jquery, os-x, vnc,...",PennApps X,58,\n Fuji is a fully-functional iOS web IDE t...,Fuji,[\nWinner\n Grand Prize\n ...
109,"[javascript, html, python, css, muse, flux, re...",Hack Cambridge,56,\n Read people's minds as they read your do...,docuflow,[\nWinner\n Hack Cambridge Peop...
34,"[java, php, mysql]",Islamabad Civic Hackathon 2016,44,\n No Food Wasted is a platform to tackle f...,nofoodwasted,[\nWinner\n Audience Favourite\...
101,"[arduino, intel-edison, ios, neopixels, deepgram]",MHacks: Refactor,44,\n LED dress that turns you into the Disney...,Sing Me A Dress,[\nWinner\n MHacks Top 5\n ...
239,"[android, ibm-watson, alchemyapi, node.js, blu...",NUI Central IBM Watson Hackathon,41,\n We automate your research\n,Fetch,[\nWinner\n 3rd Place\n ...
379,"[android, bluemix, express.js, java, node.js, ...",MHacks IV Fall 2014,37,\n Browse the internet... without data.\n,Cosmos Browser,[\nWinner\n Best Use of Twilio\...
341,"[android, samsung-gear]",Samsung Gear App Challenge - Round 2,37,\n Vector Graphics based Drawing app for Fa...,PaliPalette,[]


In [114]:
def clean_winnings(s):
    s = s.replace("\n", "")
    s = s.replace("Winner", "")
    s = s.strip()
    return s

In [116]:
df['winnings'] = df['winnings'].apply(lambda x: list(map(clean_winnings, x)))

In [131]:
CountVectorizer("3rd Prize")

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='3rd Prize',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)